Para a raspagem do Mercado Livre, foi utilizado um código para a primeira página e copiado para gerar da segunda página em diante. 
Na verdade a estrutura do codigo são iguais a mudança ocorre na construção da url.

Assim, este notebook foi divido colocando um bloco de código para cada termo pesquisado (brinquedo, escritório..) e dentro deste bloco um bloco para a primeira página e outro para as páginas seguintes (página 2 em diante).

As funções de extrair primeira página e páginas seguintes não foram terminadas, o que deixaria o código mais enxuto e legível.

A função extrai_valor_parcelas esta em aplicação, ela é usada após a raspagem para tratar a coluna Parcelas que na extração possui a quantidade de parcelas e o valor juntos, esta função cria mais 2 colunas com estes respectivos valores.


## URL de pesquisa
A estrututa do url da primeira página é simples: composto pela url da homepage mais o item pesquisado conforme abaixo:

homepage  = "https://lista.mercadolivre.com.br/"
item pesquisa = 'brinquedo'
url da pesquisa = "{homepage}{item}#D[A:{item}]"

Exemplo: https://lista.mercadolivre.com.br/notebook#D[A:notebook]

Nas paginas seguintes a estrutura muda, é inserido na estrutura a categoria do item, sendo necessário pesquisar previamente, e o número da páginas dentro do url segue um adrão diferente para cada pesquisa. Exemplo, na segunda página, aparece dentro da url  o número 49 e as para as páginas seguintes soma-se 48. Mas, para outros termos a segunda página pode ser 50, e as seguintes soma-se 51. Assim, é necessário uma pesquisa anterior para realizar a raspagem. Mas, a estrutura das urls seguem estes padrões.
Segue exemplos abaixo:

página 2 da pesquisa de brinquedo: https://lista.mercadolivre.com.br/brinquedos-hobbies/brinquedo_Desde_49_NoIndex_True  
página 3: https://lista.mercadolivre.com.br/brinquedos-hobbies/brinquedo_Desde_97_NoIndex_True


Os códigos apresentam breves comentarios para melhor entendimento.


## Patrocinados

Os produtos patrocinados aparecem em cada página, e estão em uma classe do código html diferente dos produtos normais, assim, dentro do bloco de código da Primeira página e de Páginas seguintes a primeira raspagem são nestes produtos e depois nos normais. 


# COMO PESQUISAR NESTA RASPAGEM

- Rodar a célula de importação de bibliotecas e da função extrai_valor_parcelas

- Na célula da Raspagem:
    - alterar a variável 'item' para o termo que deseja ser pesquisado
    - (se necessário) alterar a variavel 'n' para o numero que aparece na url da pagina 2 (ver URL de pesquisa mais acima).
    - (se necessário) alterar a variável incremento, verificar o incremento da url da pagina 2 para pagina 3.
    * Obs. A maioria dos itens pesquisados o n é 49 e incremento 48, mas pode haver pesquisa que n é 51 e incremento 50.
    - alterar a variável 'search_url2' para a url da página 2, dentro desta url trocar o numero 49 (ou 51, algumas vezes) para {n} (letra n entre chaves).
    - alterar a variavel 'caminho' para o endereço da pasta e nome do arquivo xlsx que ficará salvo a planilha.

Pronto! Rodar todas as células, depois de alguns minutos o arquivo xlsx aparecerá na pasta indicada no 'caminho'.

In [1]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np

# função que separa o conteudo da coluna Parcelas (numero de parcelas e valor)
# e realiza a conta do valor total das parcelas

def extrai_valor_parcelas(df):

    # separa o valor de parcela utilizando expressão regular. extrai o valor com virgula que constam após o R$
    df['Valor Parcelas'] = df['Parcelas'].str.extract(r'R\$(\d+,\d+)')
    df['Valor Parcelas'] = df['Valor Parcelas'].fillna(0)
    df['Valor Parcelas'] = df['Valor Parcelas'].apply(lambda x: float(str(x).replace(',', '.')))

    # as parcelas aparecem ao lado da letra x como: '12x'
    # expressão que separa o numeros seguidos por 'x' 
    df['N Parcelas'] = df['Parcelas'].str.extract(r'(\d+)x')
    df['N Parcelas'] = df['N Parcelas'].fillna(0)
    df['N Parcelas'] = df['N Parcelas'].apply(lambda x: int(x))

    df[' Total parcelas'] = df['Valor Parcelas'] * df['N Parcelas']

    return df

# Raspagem

In [15]:

### ALTERAR para o item que deseja buscar
item = 'meias'

# numero do url pagina 2
n = 49 

# incremento por pagina
incremento = 48

## ALTERAR o url da pagina 2, e no lugar do numero 49 colocar {n}
search_url2 = f'https://lista.mercadolivre.com.br/calcados-roupas-bolsas/moda-intima-lingerie/roupa-intima/meias-meia-calcas/meias/meias_Desde_{n}_AGE*GROUP_6725189_NoIndex_True'


## ALTERAR o caminho para exportat como planilha excel
caminho = f'C:\\Users\\prpau\\Documents\\webscraping_helpseller\\teste.xlsx'



############################################################################################################

    
base_url = "https://lista.mercadolivre.com.br/"


if ' ' in item:
    search_url = f"{base_url}{item.replace(' ', '-')}#D[A:{item.replace(' ', '%20')}]"

else:
    search_url = f"{base_url}{item}#D[A:{item}]"


# Realiza a requisição HTTP
response = requests.get(search_url)

# Verifica se a requisição foi bem-sucedida
if response.status_code != 200:
    print(f"Erro ao acessar a URL: {search_url}")

# Parsing do HTML 
soup = BeautifulSoup(response.text, 'html.parser')

# Classe html dos produtos normais
# Encontra os elementos que contêm as informações do produto
# todos os produtos tem essa classe, então usa-se o find all para trazer todos
products = soup.find_all(class_='ui-search-result__content-wrapper')


# Lista para armazenar os dados dos produtos patrocinados
products_data_patr = []

# Lista para armazenar os dados dos produtos normais
products_data = []


###########################################################################
# Classe html dos produtos patrocinados
# Encontra os elementos que contêm as informações do produto
products_patr = soup.find(class_='ui-search-layout ui-search-layout--grid')

df_patr = pd.DataFrame()

if products_patr !=  None:
    
    products_patr = products_patr.find_all('li', class_='ui-search-layout__item')
    




    for i in range(len(products_patr)):

        
        # Obtém o nome do produto
        product_name = products_patr[i].find('h2', class_='ui-search-item__title').text.strip()
        
        # Obtém o preço do produto
        product_price = products_patr[i].find('span', class_='andes-money-amount ui-search-price__part ui-search-price__part--medium andes-money-amount--cents-superscript').text

        try:
        # avaliação
            product_eval = products_patr[i].find('span', class_='ui-search-reviews__rating-number').text
        except:
            pass

        try:
        # qtde avaliações
            product_qt_eval = products_patr[i].find('span', class_='ui-search-reviews__amount').text
        except:
            product_qt_eval = ''
        
           

        try: 
            # parcelas
            product_parc = products_patr[i].find('span', class_='ui-search-item__group__element ui-search-installments ui-search-color--BLACK')

            # Extrai o texto do elemento
            product_parc = product_parc.text
        except:
            pass

        # sem juros    
        try:
            product_juros = products_patr[i].find('div', class_='ui-search-price ui-search-price--size-x-tiny ui-search-color--LIGHT_GREEN')
            product_juros = product_juros.find(text='juros')

        except:
            pass
            # desconto
        try:
            product_discount = products_patr[i].find('span', class_='ui-search-price__discount').text.strip()
        except:
            product_discount = ''


        #link
        product_link = products_patr[i].find('a', class_='ui-search-item__group__element ui-search-link__title-card ui-search-link')

        # Extrai o atributo href que está o link do produto
        product_link = product_link['href']


        # DADOS LINK

        response2 = requests.get(product_link)

        # Verifica se a requisição foi bem-sucedida
        if response2.status_code != 200:
            print(f"Erro ao acessar a URL: {search_url}")

        # Parsing do HTML
        soup2 = BeautifulSoup(response2.text, 'html.parser')


        # Nome vendedor
        try:
            
            info_element = soup2.find('span',class_ = 'ui-pdp-color--BLUE ui-pdp-family--REGULAR')

            # Extrai o vendedor 
            seller = info_element.text

        except:
            pass

        ##############################################################################################
        # posição de venda na categoria, se possui
    
        
        position = soup2.find('div', class_ = 'ui-pdp-container__col col-1 ui-pdp-container--column-right mt-16 pr-16 ui-pdp--relative')
        if position.find('div', class_ = 'ui-pdp-container__row ui-pdp-container__row--highlights') != None:
            
            position = position.find_all('a', class_ = 'ui-pdp-promotions-pill-label__target')
            position = position[1].text
        else:
            position = ''
                
 

            
        # Frete
        try:
            product_fret = soup2.find('p', class_ = 'ui-pdp-color--GREEN ui-pdp-family--SEMIBOLD ui-pdp-media__title').text
            
        except:
            product_fret = ''



        try:
        # Novo/usado e qtde vendida
            
            situation = soup2.find('div',class_ = 'ui-pdp-header__subtitle')


            # Separa as informações 'Novo' e '+500 vendidos'
            info_parts = situation.text.split('|')

            # Remove os espaços em branco ao redor de cada parte
            info_parts = [part.strip() for part in info_parts]

            product_situation = info_parts[0]
            product_sells = info_parts[1]
        except:
            pass


            # Quant. de fotos e data foto
        try:
            
            # grade que estao as fotos
            images = soup2.find_all('span',class_ = 'ui-pdp-gallery__wrapper')
            
            
            # quantidade de fotos
            images = len(images)
        except:
            pass


        # data da foto
            
        date_image = soup2.find('figure', class_ = 'ui-pdp-gallery__figure')
        
        #date_image = date_image.find('img')
        
        date_image = re.findall(r'https://\S+', str(date_image))[0]
        
        
        

    #CATEGORIAS
    ##########################################


        data_top = soup2.find_all('li',class_ = 'andes-breadcrumb__item')

        categories_list = []
        

        for cat in data_top:

            cat = cat.find('a', class_ = 'andes-breadcrumb__link')
            categories_list.append(cat['title'])
        
        
        patrocinado = 'sim'
    ###############################################

        
        # Adiciona os dados do produto à lista
        products_data_patr.append({'Nome do Produto': product_name, 'Preço': product_price, 'Patrocinado': patrocinado, 
                            'Avaliação': product_eval, 'Qtde avaliações:': product_qt_eval,'Parcelas': product_parc, 
                            'Desconto': product_discount,'Frete': product_fret, 'Link': product_link, 'Vendedor': seller, 
                            'Posição': position, 'Situação': product_situation, 'Quant. vendida': product_sells, 
                            'Categorias': categories_list,'Quant. fotos': images, 'HTML Data': date_image})

    # Cria um DataFrame com os dados dos produtos
    df_patr = pd.DataFrame(products_data_patr)






#############################################################
# Produtos nao patrocinados

for i in range(len(products)):

    
    # Obtém o nome do produto
    product_name = products[i].find('h2', class_='ui-search-item__title').text.strip()
    
    # Obtém o preço do produto
    product_price = products[i].find('span', class_='andes-money-amount ui-search-price__part ui-search-price__part--medium andes-money-amount--cents-superscript').text.strip()

    # se patrocinado caso, pagina seja com produtos na vertical
    product_patr = products[i].find('label', class_='ui-search-styled-label ui-search-item__pub-label')
    

    
    if product_patr == None:
        patrocinado = 'nao'

    else:
        patrocinado = 'sim'
    
    
    
    
    try:
    # avaliação
        product_eval = products[i].find('span', class_='ui-search-reviews__rating-number').text.strip()
    except:
        pass

    try:
    # qtde avaliações
        product_qt_eval = products[i].find('span', class_='ui-search-reviews__amount').text.strip()
    except:
        pass
    
    #frete gratis
    try:
        product_fret = products[i].find('span', class_='ui-pb-highlight').text.strip()
    except:
        product_fret = ''    

    try: 
        # parcelas
        product_parc = products[i].find('span', class_='ui-search-item__group__element ui-search-installments ui-search-color--BLACK')

        # Extrai o texto do elemento
        product_parc = product_parc.get_text(strip=True)
    except:
        pass

    # sem juros    
    try:
        product_juros = products[i].find('div', class_='ui-search-price ui-search-price--size-x-tiny ui-search-color--LIGHT_GREEN')
        product_juros = product_juros.find(text='juros')

    except:
        pass
        # desconto
    try:
        product_discount = products[i].find('span', class_='ui-search-price__discount').text.strip()
    except:
        product_discount = ''


    #link
    product_link = products[i].find('a', class_='ui-search-item__group__element ui-search-link__title-card ui-search-link')

    # Extrai o atributo href do elemento
    product_link = product_link['href']


    # DADOS LINK

    response2 = requests.get(product_link)

    # Verifica se a requisição foi bem-sucedida
    if response2.status_code != 200:
        print(f"Erro ao acessar a URL: {search_url}")

    # Parsing do HTML
    soup2 = BeautifulSoup(response2.text, 'html.parser')


    # Nome vendedor
    try:
        #info_element = soup2.find('div', class_='ui-pdp-container__row ui-pdp--relative ui-pdp-with--separator--fluid pb-40')
        #info_element = info_element.find('div',class_ = 'ui-pdp--sticky-wrapper ui-pdp--sticky-wrapper-right')
        #info_element = info_element.find('div',class_ = 'ui-pdp-seller mb-20 mt-20 ui-pdp-seller__with-logo')
        info_element = soup2.find('span',class_ = 'ui-pdp-color--BLUE ui-pdp-family--REGULAR')

        # Extrai o vendedor 
        seller = info_element.text

    except:
        pass
    
    
    # posição de venda na categoria, se possui
    
   
    position = soup2.find('div', class_ = 'ui-pdp-container__col col-1 ui-pdp-container--column-right mt-16 pr-16 ui-pdp--relative')
    if position.find('div', class_ = 'ui-pdp-container__row ui-pdp-container__row--highlights') != None:
        
        position = position.find_all('a', class_ = 'ui-pdp-promotions-pill-label__target')
        position = position[1].text
    else:
        position = ''
            
    

    # Frete
    try:
        product_fret = soup2.find('p', class_ = 'ui-pdp-color--GREEN ui-pdp-family--SEMIBOLD ui-pdp-media__title').text
        
    except:
        product_fret = ''


    try:
    # Novo/usado e qtde vendida
        #situation = soup2.find('div', class_='ui-pdp-container__col col-2 mr-32')
        situation = soup2.find('div',class_ = 'ui-pdp-header__subtitle')


        # Separa as informações 'Novo' e '+500 vendidos'
        info_parts = situation.text.split('|')

        # Remove os espaços em branco ao redor de cada parte
        info_parts = [part.strip() for part in info_parts]

        product_situation = info_parts[0]
        product_sells = info_parts[1]
    except:
        pass

        # Quant. de fotos
    try:
        # qtde fotos
        #images = soup2.find('div', class_='ui-pdp-container__row ui-pdp-with--separator--fluid ui-pdp-with--separator--40')
        #images = images.find('div',class_ = 'ui-pdp-container__col col-2 ui-pdp--relative')

        #images= images.find('div',class_ = 'ui-pdp-gallery__column')
        images = soup2.find_all('span',class_ = 'ui-pdp-gallery__wrapper')

        images = len(images)
    except:
        pass

    
     # data da foto
            
    date_image = soup2.find('figure', class_ = 'ui-pdp-gallery__figure')
    
    
    
    date_image = re.findall(r'https://\S+', str(date_image))[0]
    
    
    
    # categorias
    data_top = soup2.find_all('li',class_ = 'andes-breadcrumb__item')

    categories_list = []
    

    for cat in data_top:

        cat = cat.find('a', class_ = 'andes-breadcrumb__link')
        categories_list.append(cat['title'])

    
###############################################

    
    # Adiciona os dados do produto à lista
    # Adiciona os dados do produto à lista
    products_data.append({'Nome do Produto': product_name, 'Preço': product_price, 'Patrocinado': patrocinado, 
                          'Avaliação': product_eval, 'Qtde avaliações:': product_qt_eval,'Parcelas': product_parc, 
                          'Desconto': product_discount,'Frete': product_fret, 'Link': product_link, 'Vendedor': seller, 
                          'Posição': position, 'Situação': product_situation, 'Quant. vendida': product_sells, 
                          'Categorias': categories_list,'Quant. fotos': images, 'HTML Data': date_image})

# Cria um DataFrame com os dados dos produtos
df = pd.DataFrame(products_data)


In [20]:
# DataFrame vazio para armazenar todos os dados um para patrocinados e outro não patrocinado
all_products_df = pd.DataFrame()
all_products_df_patr = pd.DataFrame()

for i in range(1, 5):
    

    # Realiza a requisição HTTP
    response = requests.get(search_url2)

    # Verifica se a requisição foi bem-sucedida
    if response.status_code != 200:
        print(f"Erro ao acessar a URL: {search_url}")
        continue

    # Parsing do HTML
    soup = BeautifulSoup(response.text, 'html.parser')

    # Lista para armazenar os dados dos produtos desta iteração
    products_data = []

    # Encontra os elementos que contêm as informações do produto
    products = soup.find_all(class_='ui-search-result__content-wrapper')



    
    all_products_data_patr = []


    # produtos patrocinados
    
    products_patr = soup.find(class_='ui-search-layout ui-search-layout--grid')
    

    if products_patr !=  None:
        
        products_patr = products_patr.find_all('li', class_='ui-search-layout__item')

        for x in range(len(products_patr)):

            
            # Obtém o nome do produto
            product_name = products_patr[x].find('h2', class_='ui-search-item__title').text.strip()
            
            # Obtém o preço do produto
            product_price = products_patr[x].find('span', class_='andes-money-amount ui-search-price__part ui-search-price__part--medium andes-money-amount--cents-superscript').text.strip()

            try:
            # avaliação
                product_eval = products_patr[x].find('span', class_='ui-search-reviews__rating-number').text.strip()
            except:
                pass

            try:
            # qtde avaliações
                product_qt_eval = products_patr[x].find('span', class_='ui-search-reviews__amount').text.strip()
            except:
                pass
            
            #frete gratis
            try:
                product_fret = products_patr[x].find('span', class_='ui-pb-highlight').text.striP()
            except:
                product_fret = ''    

            try: 
                # parcelas
                product_parc = products_patr[x].find('span', class_='ui-search-item__group__element ui-search-installments ui-search-color--BLACK')

                # Extrai o texto do elemento
                product_parc = product_parc.get_text(strip=True)
            except:
                pass

            # sem juros    
            try:
                product_juros = products_patr[x].find('div', class_='ui-search-price ui-search-price--size-x-tiny ui-search-color--LIGHT_GREEN')
                product_juros = product_juros.find(text='juros')

            except:
                pass
                # desconto
            try:
                product_discount = products_patr[x].find('span', class_='ui-search-price__discount').text.strip()
            except:
                product_discount = ''


            #link
            product_link = products_patr[x].find('a', class_='ui-search-item__group__element ui-search-link__title-card ui-search-link')

            # Extrai o atributo href do elemento
            product_link = product_link['href']


            # DADOS LINK

            response2 = requests.get(product_link)

            # Verifica se a requisição foi bem-sucedida
            if response2.status_code != 200:
                print(f"Erro ao acessar a URL: {product_link}")

            # Parsing do HTML
            soup2 = BeautifulSoup(response2.text, 'html.parser')


            # Nome vendedor
            try:
                #info_element = soup2.find('div', class_='ui-pdp-container__row ui-pdp--relative ui-pdp-with--separator--fluid pb-40')
                #info_element = info_element.find('div',class_ = 'ui-pdp--sticky-wrapper ui-pdp--sticky-wrapper-right')
                #info_element = info_element.find('div',class_ = 'ui-pdp-seller mb-20 mt-20 ui-pdp-seller__with-logo')
                info_element = soup2.find('span',class_ = 'ui-pdp-color--BLUE ui-pdp-family--REGULAR')

                # Extrai o vendedor 
                seller = info_element.text

            except:
                seller= ''

            # posição de venda na categoria, se possui
            
            try:
                position = soup2.find('div', class_ = 'ui-pdp-container__col col-1 ui-pdp-container--column-right mt-16 pr-16 ui-pdp--relative')
                if position.find('div', class_ = 'ui-pdp-container__row ui-pdp-container__row--highlights') != None:
                    
                    position = position.find_all('a', class_ = 'ui-pdp-promotions-pill-label__target')
                    position = position[1].text
                else:
                    position = ''
                    
            except:
                position = ''

            # Frete
            try:
                product_fret = soup2.find('p', class_ = 'ui-pdp-color--GREEN ui-pdp-family--SEMIBOLD ui-pdp-media__title').text
                
            except:
                product_fret = ''


            try:
            # Novo/usado e qtde vendida
                #situation = soup2.find('div', class_='ui-pdp-container__col col-2 mr-32')
                situation = soup2.find('div',class_ = 'ui-pdp-header__subtitle')


                # Separa as informações 'Novo' e '+500 vendidos'
                info_parts = situation.text.split('|')

                # Remove os espaços em branco ao redor de cada parte
                info_parts = [part.strip() for part in info_parts]

                product_situation = info_parts[0]
                product_sells = info_parts[1]
            except:
                pass

                # Quant. de fotos
            try:
                # qtde fotos
                #images = soup2.find('div', class_='ui-pdp-container__row ui-pdp-with--separator--fluid ui-pdp-with--separator--40')
                #images = images.find('div',class_ = 'ui-pdp-container__col col-2 ui-pdp--relative')

                #images= images.find('div',class_ = 'ui-pdp-gallery__column')
                images = soup2.find_all('span',class_ = 'ui-pdp-gallery__wrapper')

                images = len(images)
            except:
                pass
            
            
            
             # data da foto
            date_image = soup2.find('figure', class_ = 'ui-pdp-gallery__figure')
            
            #date_image = date_image.find('img')
            
            date_image = re.findall(r'https://\S+', str(date_image))



            # categorias
            data_top = soup2.find_all('li',class_ = 'andes-breadcrumb__item')

            categories_list = []
            

            for cat in data_top:

                cat = cat.find('a', class_ = 'andes-breadcrumb__link')
                categories_list.append(cat['title'])
        
    ###############################################

        
            
        
            patrocinado = 'sim'
            
            # Adiciona os dados do produto à lista
            all_products_data_patr.append({'Nome do Produto': product_name, 'Preço': product_price, 'Patrocinado': patrocinado, 
                            'Avaliação': product_eval, 'Qtde avaliações:': product_qt_eval,'Parcelas': product_parc, 
                            'Desconto': product_discount,'Frete': product_fret, 'Link': product_link, 'Vendedor': seller, 
                            'Posição': position, 'Situação': product_situation, 'Quant. vendida': product_sells, 
                            'Categorias': categories_list,'Quant. fotos': images, 'HTML Data': date_image})

    # Cria um DataFrame com os dados dos produtos

        patr_temp_df = pd.DataFrame(all_products_data_patr)
        all_products_df_patr = pd.concat([all_products_df_patr, patr_temp_df], ignore_index=True)
        



##########################################################################################################################3






    for product in products:
        # Obtém o nome do produto
        product_name = product.find('h2', class_='ui-search-item__title').text.strip()

        #preço
        try:
            product_price = product.find('span', class_='andes-money-amount ui-search-price__part ui-search-price__part--medium andes-money-amount--cents-superscript').text.strip()
        except:
            product_price = product.find('span', class_='andes-money-amount__fraction').text.strip()


        # se patrocinado caso, pagina seja com produtos na vertical
        product_patr = products[i].find('label', class_='ui-search-styled-label ui-search-item__pub-label')
        

        
        

        if product_patr == None:
            patrocinado = 'nao'
            
        else:
            patrocinado = 'sim'
            


        try:
    # avaliação
            product_eval = product.find('span', class_='ui-search-reviews__rating-number').text.strip()
        except:
            pass

        try:
        # qtde avaliações
            product_qt_eval = product.find('span', class_='ui-search-reviews__amount').text.strip()
        except:
            pass
        
        #frete gratis
        try:
            product_fret = product.find('span', class_='ui-pb-highlight').text.strip()
        except:
            product_fret = ''    


        try: 
        # parcelas
            product_parc = product.find('span', class_='ui-search-item__group__element ui-search-installments ui-search-color--BLACK')

            # Extrai o texto do elemento
            product_parc = product_parc.get_text(strip=True)
        except:
            pass

        # sem juros    
        try:
            product_juros = products.find('span', class_='ui-search-price ui-search-price--size-x-tiny ui-search-color--LIGHT_GREEN')
            product_juros = product_juros.find(text='juros')

        except:
            pass
        # desconto
        try:
            product_discount = product.find('span', class_='ui-search-price__discount').text.strip()
        except:
            product_discount = ''


        

        #link
        product_link = product.find('a', class_='ui-search-item__group__element ui-search-link__title-card ui-search-link')

        # Extrai o atributo href do elemento
        product_link = product_link['href']


        # DADOS LINK

        response2 = requests.get(product_link)

        # Verifica se a requisição foi bem-sucedida
        if response2.status_code != 200:
            print(f"Erro ao acessar a URL: {search_url}")

        # Parsing do HTML
        soup2 = BeautifulSoup(response2.text, 'html.parser')

        # Vendedor
        try:
            #info_element = soup2.find('div', class_='ui-pdp-container__row ui-pdp--relative ui-pdp-with--separator--fluid pb-40')
            #info_element = info_element.find('div',class_ = 'ui-pdp--sticky-wrapper ui-pdp--sticky-wrapper-right')
            #info_element = info_element.find('div',class_ = 'ui-pdp-seller mb-20 mt-20 ui-pdp-seller__with-logo')
            info_element = soup2.find('span',class_ = 'ui-pdp-color--BLUE ui-pdp-family--REGULAR').text

            # Extrai o texto do elemento
            seller = info_element
        except:
            pass

        
        # posição de venda na categoria, se possui
    
        try:
            position = soup2.find('div', class_ = 'ui-pdp-container__col col-1 ui-pdp-container--column-right mt-16 pr-16 ui-pdp--relative')
            if position.find('div', class_ = 'ui-pdp-container__row ui-pdp-container__row--highlights') != None:
                
                position = position.find_all('a', class_ = 'ui-pdp-promotions-pill-label__target')
                position = position[1].text

            else:
                position = ''
                
        except:
            position = ''

        # Frete
        try:
            product_fret = soup2.find('p', class_ = 'ui-pdp-color--GREEN ui-pdp-family--SEMIBOLD ui-pdp-media__title').text
            
        except:
            product_fret = ''
            

            # Novo/usado e qtde vendida
        try:
        
            #situation = soup2.find('div', class_='ui-pdp-container__col col-2 mr-32')
            situation = soup2.find('div',class_ = 'ui-pdp-header__subtitle')


            # Separa as informações 'Novo' e '+500 vendidos'
            info_parts = situation.text.split('|')

            # Remove os espaços em branco ao redor de cada parte
            info_parts = [part.strip() for part in info_parts]

            product_situation = info_parts[0]
            product_sells = info_parts[1]
        except:
            pass

            # Quant. de fotos
        try:
            
            #images = soup2.find('div', class_='ui-pdp-container__row ui-pdp-with--separator--fluid ui-pdp-with--separator--40')
            #images = images.find('div',class_ = 'ui-pdp-container__col col-2 ui-pdp--relative')

            #images= images.find('div',class_ = 'ui-pdp-gallery__column')
            images = soup2.find_all('span',class_ = 'ui-pdp-gallery__wrapper')

            images = len(images)
        except:
            pass




         # data da foto
            
        date_image = soup2.find('figure', class_ = 'ui-pdp-gallery__figure')
        
        #date_image = date_image.find('img')
        
        date_image = re.findall(r'https://\S+', str(date_image))


#####################################

        # categorias
        data_top = soup2.find_all('li',class_ = 'andes-breadcrumb__item')

        categories_list = []
        

        for cat in data_top:

            cat = cat.find('a', class_ = 'andes-breadcrumb__link')
            categories_list.append(cat['title'])
    
###############################################

        # Adiciona os dados do produto à lista
        products_data.append({'Nome do Produto': product_name, 'Preço': product_price, 'Patrocinado': patrocinado, 
                          'Avaliação': product_eval, 'Qtde avaliações:': product_qt_eval,'Parcelas': product_parc, 
                          'Desconto': product_discount,'Frete': product_fret, 'Link': product_link, 'Vendedor': seller, 
                          'Posição': position, 'Situação': product_situation, 'Quant. vendida': product_sells, 
                          'Categorias': categories_list,'Quant. fotos': images, 'HTML Data': date_image})



####    # Incrementa a posição conforme a página seguinte
    
    n = n + incremento
    # Cria um DataFrame temporário com os dados dos produtos desta iteração
    temp_df = pd.DataFrame(products_data)

    # Concatena o DataFrame temporário ao DataFrame principal
    all_products_df = pd.concat([all_products_df, temp_df], ignore_index=True)




# juntando os dataframes
df_mercado_brinquedo = pd.concat([df, df_patr,all_products_df_patr,all_products_df], join = 'outer')
df_mercado_brinquedo = extrai_valor_parcelas(df_mercado_brinquedo)
df_mercado_brinquedo.drop(['Parcelas'], axis = 1, inplace = True)

df_mercado_brinquedo['Categoria'] = item



df_mercado_brinquedo['Categoria0'] = 0
df_mercado_brinquedo['Categoria1'] = 0
df_mercado_brinquedo['Categoria2'] = 0
df_mercado_brinquedo['Categoria3'] = 0
df_mercado_brinquedo['Categoria4'] = 0
df_mercado_brinquedo['Categoria5'] = 0
df_mercado_brinquedo['Categoria6'] = 0

for i in range(len(df_mercado_brinquedo['Categorias'])):

    for x in range(len(df_mercado_brinquedo['Categorias'].iloc[i])):
        df_mercado_brinquedo[f'Categoria{x}'].iloc[i] = df_mercado_brinquedo['Categorias'].iloc[i][x]

        
        
for i in range(len(df_mercado_brinquedo['Categorias'])):

        for x in range(1,7):
            if df_mercado_brinquedo[f'Categoria{x}'].iloc[i] == 0:
                df_mercado_brinquedo[f'Categoria{x}'].iloc[i] = df_mercado_brinquedo[f'Categoria{x-1}'].iloc[i]




### Exportando o dataframe para planlha excel
df_mercado_brinquedo.to_excel(caminho, index=False)


Erro ao acessar a URL: https://lista.mercadolivre.com.br/meias#D[A:meias]
Erro ao acessar a URL: https://lista.mercadolivre.com.br/meias#D[A:meias]
Erro ao acessar a URL: https://lista.mercadolivre.com.br/meias#D[A:meias]


C:\Users\prpau\AppData\Local\Temp\ipykernel_13620\49187481.py:448: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_mercado_brinquedo[f'Categoria{x}'].iloc[i] = df_mercado_brinquedo['Categorias'].iloc[i][x]


In [81]:
### Exportando o dataframe para planlha excel
df_mercado_brinquedo.to_excel(caminho, index=False)
